In [1]:
# install beauitfulsoup - its' a library used to pass html code to find the elements inside it


#!pip install bs4

In [2]:
# requests send from local pc to the website


#!pip install requests

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [4]:
# Import the necessary libraries
from bs4 import BeautifulSoup
import requests

url = "https://www.amazon.com/s?k=laptops&crid=4TRJMYQROWKF&sprefix=laptops%2Caps%2C99&ref=nb_sb_noss_1"

# Headers for request

headers = ({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5',
    'Referer': 'https://www.cvs.com/'
})

# Make a request to the webpage
webpage = requests.get(url, headers = headers)

# type(webpage.content)

# Create a BeautifulSoup object
html_main_page = BeautifulSoup(webpage.content, "html.parser")

# Find all the links with the specified attributes
links_from_main_page = html_main_page.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

# Create an empty list to store the links
laptop_links = []

# Iterate through the links and retrieve the href attribute
for link in links_from_main_page:
    href = link.get('href')
    laptop_link = "https://www.amazon.com" + href
    laptop_links.append(laptop_link)

# Print all the links
# for link in laptop_links:
    #print(link)


In [5]:
pd.DataFrame(laptop_links).shape

(36, 1)

In [6]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

product_details = []  # Initialize the list

for laptop_link in laptop_links:
    try:
        # Make a request to the laptop page
        laptop_page = requests.get(laptop_link, headers=headers)

        # Check if the URL has been redirected
        if 'picassoRedirect.html' in laptop_page.url:
            # Extract the redirected URL from the "url" parameter
            redirected_url = re.search(r'url=(.*?)&', laptop_page.url).group(1)
            
            # Make a new request to the redirected URL
            laptop_page = requests.get(redirected_url, headers=headers)

        # Create a BeautifulSoup object
        laptop_page_html = BeautifulSoup(laptop_page.content, "html.parser")

        # Find the product title
        product_title = laptop_page_html.find("span", attrs={"id": 'productTitle'}).text.strip()

        # Find the product price
        product_price = laptop_page_html.find("span", attrs={'class': 'a-offscreen'}).text.strip()

        # Find the product rating
        product_rating = laptop_page_html.find("span", attrs={'class': 'a-icon-alt'}).text.strip()

        # Find additional details
        html = laptop_page_html.find("div", attrs={'id': "poExpander"})
        key_elements = html.find_all('span', class_='a-size-base a-text-bold')
        value_elements = html.find_all('span', class_='a-size-base po-break-word')
        additional_details = {}

        # Iterate over the key-value elements and extract their values
        for key_element, value_element in zip(key_elements, value_elements):
            key = key_element.text.strip().lower().replace(' ', '_')
            value = value_element.text.strip()
            additional_details[key] = value

        # Append the product details to the list
        product_details.append({
            'title': product_title,
            'price': product_price,
            'rating': product_rating,
            **additional_details
        })
    except Exception as e:
        print(f"Error occurred while processing link: {laptop_link}")
        print(f"Error message: {str(e)}")
        continue

# Create a dataframe from the product details list
df = pd.DataFrame(product_details)

# Print the dataframe
#print(df)


Error occurred while processing link: https://www.amazon.comhttps://aax-us-iad.amazon.com/x/c/RA9TEYXgo-ALhskIDv3pW5YAAAGIYRRETQEAAAH2AQBvbm9fdHhuX2JpZDcgICBvbm9fdHhuX2ltcDEgICDh2EP5/https://www.amazon.com/HP-Display-i3-1215U-Graphics-15-dy5599nr/dp/B0BVD8LZQL/ref=sxin_18_sbv_search_btf?content-id=amzn1.sym.b000e0a0-9e93-480f-bf78-a83c8136dfcb%3Aamzn1.sym.b000e0a0-9e93-480f-bf78-a83c8136dfcb&crid=4TRJMYQROWKF&cv_ct_cx=laptops&keywords=laptops&pd_rd_i=B0BVD8LZQL&pd_rd_r=165beb64-635b-478d-9c4e-c2e27affbff7&pd_rd_w=RfP4j&pd_rd_wg=4CeLI&pf_rd_p=b000e0a0-9e93-480f-bf78-a83c8136dfcb&pf_rd_r=03WHFY1479SGC7EQ2KRG&qid=1685255898&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=laptops%2Caps%2C99&sr=1-1-5190daf0-67e3-427c-bea6-c72c1df98776
Error message: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: //aax-us-iad.amazon.com/x/c/RA9TEYXgo-ALhskIDv3pW5YAAAGIYRRETQEAAAH2AQBvbm9fdHhuX2JpZDcgICBvbm9fdHhuX2ltcDEgICDh2EP5/https://www.amazon.com/HP-Display-i3-12

In [7]:
df.to_csv('laptop_details.csv', index=False)